In [1]:
import rich

from sdRDM import DataModel

### Code Generation

In [2]:
# There are two options for using your model
# 1. Generate code from the model via CLI and use it in your project

!sdRDM generate --path ./model.md --out . --name MyLibrary

# This will generate a library called MyLibrary in the current directory
# You can use it like any other python library

In [3]:
# 2. You can use the model directly in your code without generating code
#
# This will result in an object that hold all your class definitions

lib = DataModel.from_markdown("./model.md")
lib

Objects MyObject, AnotherObject

### Populating data models

In [4]:
# Using an object and populating it with data

dataset = lib.MyObject(
    attribute="value",
    mandatory_attribute=100.0,
    array_attribute=[1, 2, 3],
)

# To inspect the content of the object, simply print it
# to receive a tree representation of the object

print(dataset)

MyObject
├── id = b03ec777-dda3-4f9f-809d-3cae15ac98b3
├── attribute = value
├── mandatory_attribute = 100.0
├── array_attribute = [1.0, 2.0, 3.0, ...]
└── object_attribute
    └── AnotherObject
        ├── id = 7acd48e8-9967-430a-b87b-0bb6d7d02c9e
        └── small_type
            └── SmallType
                └── id = eb905685-23fa-4705-a14b-cea835cda321



In [5]:
# As you might have noticed, the attribute `object:attribute` is already added.
# This is due to the fact that whenever a sub object contains no mandatory attributes,
# it is added by default. No worries, upon export empty objects will not be included.

# You can now easily set attributes of the object

dataset.object_attribute.small_type.name = "Small Object"
dataset.object_attribute.small_type.value = 1.0

# And inspect the result ...

print(dataset)

MyObject
├── id = b03ec777-dda3-4f9f-809d-3cae15ac98b3
├── attribute = value
├── mandatory_attribute = 100.0
├── array_attribute = [1.0, 2.0, 3.0, ...]
└── object_attribute
    └── AnotherObject
        ├── id = 7acd48e8-9967-430a-b87b-0bb6d7d02c9e
        └── small_type
            └── SmallType
                ├── id = eb905685-23fa-4705-a14b-cea835cda321
                ├── value = 1.0
                └── name = Small Object



In [6]:
# We are still missing the `multiple_object_attribute` which is a reference 
# to another object and also an array attribute

# The generated code containes adder-methods to simply add objects
# without the need of creatiung the object first

added_object = dataset.add_to_multiple_object_attribute(
    small_type={
        "value": 100.0,
        "name": "I am a small object"
    }
)

print(">>> The added object", added_object, sep="\n\n")

# This method will also return a reference to the newly added object
# which you can edit directly or add to other things in your code

added_object.small_type.value = 200.0

print(">>> The dataset", dataset, sep="\n\n")

>>> The added object

AnotherObject
├── id = 6de37403-4544-4d7e-b791-7255aed0df09
└── small_type
    └── SmallType
        ├── id = 3a96af7e-eee1-4876-8ddf-dd097ffe8002
        ├── value = 100.0
        └── name = I am a small object

>>> The dataset

MyObject
├── id = b03ec777-dda3-4f9f-809d-3cae15ac98b3
├── attribute = value
├── mandatory_attribute = 100.0
├── array_attribute = [1.0, 2.0, 3.0, ...]
├── object_attribute
│   └── AnotherObject
│       ├── id = 7acd48e8-9967-430a-b87b-0bb6d7d02c9e
│       └── small_type
│           └── SmallType
│               ├── id = eb905685-23fa-4705-a14b-cea835cda321
│               ├── value = 1.0
│               └── name = Small Object
└── multiple_object_attribute
    └── 0
        └── AnotherObject
            ├── id = 6de37403-4544-4d7e-b791-7255aed0df09
            └── small_type
                └── SmallType
                    ├── id = 3a96af7e-eee1-4876-8ddf-dd097ffe8002
                    ├── value = 200.0
                    └── name = 

### Special case: Units

In [25]:
# Out of the box the library supports units that can be used in your model
# These can be initialized as a string, but will later on be decomposed into
# base units to allow for easy conversion.

# sdRDM utilizes astropy.units for unit handling and conversion

dataset.some_unit = "mmol / l"

print(dataset.some_unit)

# You can access the astropy unit object via the `_unit` attribute
print("Nice looking LaTeX representation:")
dataset.some_unit._unit

Unit
├── id = 4eabbc7c-df20-48d7-8801-5a05c49c6249
├── name = mmol / l
└── bases
    ├── 0
    │   └── BaseUnit
    │       ├── scale = 0.001
    │       ├── kind = mol
    │       └── exponent = 1.0
    └── 1
        └── BaseUnit
            ├── scale = 1.0
            ├── kind = l
            └── exponent = -1.0

Nice looking LaTeX representation:


Unit("mmol / l")

### Model navigation and filtering

In [9]:
# This has grown a lot, doesnt it? But we are not done yet. Navigating through
# heavily nested objects can be a pain. Therefore, sdRDM provides a simple method
# to get an object by its path, but first, lets see how we can find all paths!

dataset.paths(leaves=True)

[Path(/id),
 Path(/attribute),
 Path(/mandatory_attribute),
 Path(/array_attribute/0),
 Path(/array_attribute/1),
 Path(/array_attribute/2),
 Path(/object_attribute/id),
 Path(/object_attribute/small_type/id),
 Path(/object_attribute/small_type/value),
 Path(/object_attribute/small_type/name),
 Path(/multiple_object_attribute/0/id),
 Path(/multiple_object_attribute/0/small_type/id),
 Path(/multiple_object_attribute/0/small_type/value),
 Path(/multiple_object_attribute/0/small_type/name),
 Path(/some_unit/id),
 Path(/some_unit/name),
 Path(/some_unit/bases/0/scale),
 Path(/some_unit/bases/0/kind),
 Path(/some_unit/bases/0/exponent),
 Path(/some_unit/bases/1/scale),
 Path(/some_unit/bases/1/kind),
 Path(/some_unit/bases/1/exponent)]

In [10]:
# Now that we know all paths, we can use them to get the object we want

dataset.get("/multiple_object_attribute/0/small_type/name")

'I am a small object'

In [11]:
# Nice! But if you look closely you will notice that each path also
# contains a number, if the object is an array. This is a very precise
# way of getting an object, but it is also very verbose. Therefore, you can
# also use so called `meta_paths` to get all the objects you want.

dataset.meta_paths()

['array_attribute',
 'attribute',
 'id',
 'mandatory_attribute',
 'multiple_object_attribute',
 'multiple_object_attribute/id',
 'multiple_object_attribute/small_type',
 'multiple_object_attribute/small_type/id',
 'multiple_object_attribute/small_type/name',
 'multiple_object_attribute/small_type/value',
 'object_attribute',
 'object_attribute/id',
 'object_attribute/small_type',
 'object_attribute/small_type/id',
 'object_attribute/small_type/name',
 'object_attribute/small_type/value',
 'some_unit',
 'some_unit/bases',
 'some_unit/bases/exponent',
 'some_unit/bases/kind',
 'some_unit/bases/scale',
 'some_unit/id',
 'some_unit/name']

In [12]:
# Lets add another object to the array and try to get all small objects
# since we are not interested in `AnotherObject` - its context

_ = dataset.add_to_multiple_object_attribute(
    small_type={
        "name": "Another small object",
        "value": 100.0
    }
)

# Now we can get all small objects by using the meta path

dataset.get("multiple_object_attribute/small_type")

[SmallType(id='3a96af7e-eee1-4876-8ddf-dd097ffe8002', value=200.0, name='I am a small object'),
 SmallType(id='1bd3eada-2beb-4598-861b-76caeefd57f2', value=100.0, name='Another small object')]

In [18]:
# Awesome! Saved a lot of code and time. But what if we want to get a specific object?
# Lets try to extract only those that contain a certain ID

dataset.get(
    "multiple_object_attribute/small_type",    
    attribute="name",
    target="I am a small object"
)

[SmallType(id='3a96af7e-eee1-4876-8ddf-dd097ffe8002', value=200.0, name='I am a small object')]

### Export and import of datasets

In [19]:
# Okay, now you know most of the things to work with sdRDM objects.
# Finally, lets export the object to a file of interest. sdRDM supports
# a variety of formats, including JSON, YAML, XML and HDF5 out of the box.

# ! We omit HDF5 here, since its hard to print it in a nice way

# Lets export the object to JSON
with open("dataset.json", "w") as f:
    json_data = dataset.json(warn=False)
    f.write(json_data)
    
    rich.print(json_data)

{
  "id": "b03ec777-dda3-4f9f-809d-3cae15ac98b3",
  "attribute": "value",
  "mandatory_attribute": 100.0,
  "array_attribute": [
    1.0,
    2.0,
    3.0
  ],
  "object_attribute": {
    "id": "7acd48e8-9967-430a-b87b-0bb6d7d02c9e",
    "small_type": {
      "id": "eb905685-23fa-4705-a14b-cea835cda321",
      "value": 1.0,
      "name": "Small Object"
    }
  },
  "multiple_object_attribute": [
    {
      "id": "6de37403-4544-4d7e-b791-7255aed0df09",
      "small_type": {
        "id": "3a96af7e-eee1-4876-8ddf-dd097ffe8002",
        "value": 200.0,
        "name": "I am a small object"
      }
    },
    {
      "id": "6e6be1ff-4fc0-483e-9980-5093f9ff21cb",
      "small_type": {
        "id": "1bd3eada-2beb-4598-861b-76caeefd57f2",
        "value": 100.0,
        "name": "Another small object"
      }
    }
  ],
  "some_unit": {
    "id": "8a7979db-eaa9-42e2-8cc2-ce7a64dcda99",
    "name": "mmol / l",
    "bases": [
      {
        "scale": 0.001,
        "kind": "mol",
        "exponent": 1.0
      },
      {
        "scale": 1.0,
        "kind": "l",
        "exponent": -1.0
      }
    ]
  }
}

In [20]:
# Or to YAML
with open("dataset.yaml", "w") as f:
    yaml_data = dataset.yaml(warn=False)
    f.write(yaml_data)
    
    rich.print(yaml_data)

id: b03ec777-dda3-4f9f-809d-3cae15ac98b3
attribute: value
mandatory_attribute: 100.0
array_attribute:
  - 1.0
  - 2.0
  - 3.0
object_attribute:
  id: 7acd48e8-9967-430a-b87b-0bb6d7d02c9e
  small_type:
    id: eb905685-23fa-4705-a14b-cea835cda321
    value: 1.0
    name: Small Object
multiple_object_attribute:
  - id: 6de37403-4544-4d7e-b791-7255aed0df09
    small_type:
      id: 3a96af7e-eee1-4876-8ddf-dd097ffe8002
      value: 200.0
      name: I am a small object
  - id: 6e6be1ff-4fc0-483e-9980-5093f9ff21cb
    small_type:
      id: 1bd3eada-2beb-4598-861b-76caeefd57f2
      value: 100.0
      name: Another small object
some_unit:
  id: 8a7979db-eaa9-42e2-8cc2-ce7a64dcda99
  name: mmol / l
  bases:
    - scale: 0.001
      kind: mol
      exponent: 1.0
    - scale: 1.0
      kind: l
      exponent: -1.0

In [23]:
# Or to XML
with open("dataset.xml", "w") as f:
    xml_data = dataset.xml()
    f.write(xml_data)
    
    rich.print(xml_data)

<?xml version='1.0' encoding='UTF-8'?>
<MyObject id="b03ec777-dda3-4f9f-809d-3cae15ac98b3">
  <string>value</string>
  <float>100.0</float>
  <array_attribute>
    <float>1.0</float>
    <float>2.0</float>
    <float>3.0</float>
  </array_attribute>
  <AnotherObject id="7acd48e8-9967-430a-b87b-0bb6d7d02c9e">
    <SmallType id="eb905685-23fa-4705-a14b-cea835cda321">
      <float>1.0</float>
      <string>Small Object</string>
    </SmallType>
  </AnotherObject>
  <multiple_object_attribute>
    <AnotherObject id="6de37403-4544-4d7e-b791-7255aed0df09">
      <SmallType id="3a96af7e-eee1-4876-8ddf-dd097ffe8002">
        <float>200.0</float>
        <string>I am a small object</string>
      </SmallType>
    </AnotherObject>
    <AnotherObject id="6e6be1ff-4fc0-483e-9980-5093f9ff21cb">
      <SmallType id="1bd3eada-2beb-4598-861b-76caeefd57f2">
        <float>100.0</float>
        <string>Another small object</string>
      </SmallType>
    </AnotherObject>
  </multiple_object_attribute>
  <Unit xmlns="https://www.github.com/software-driven-rdm" id="8a7979db-eaa9-42e2-8cc2-ce7a64dcda99" name="mmol / 
l">
    <BaseUnit scale="0.001" kind="mol" exponent="1.0"/>
    <BaseUnit scale="1.0" kind="l" exponent="-1.0"/>
  </Unit>
</MyObject>

In [24]:
# Last but not least, you can read in data from a file and convert it to an object
# using the object of interest and the dedicated method of the format

# We will use XML since it requires some special treatment. All others can be read
# without byte reading!

with open("dataset.xml", "rb") as f:
    parsed = lib.MyObject.from_xml(f)
    
print(parsed)

MyObject
├── id = b03ec777-dda3-4f9f-809d-3cae15ac98b3
├── attribute = value
├── mandatory_attribute = 100.0
├── array_attribute = [1.0, 2.0, 3.0, ...]
├── object_attribute
│   └── AnotherObject
│       ├── id = 7acd48e8-9967-430a-b87b-0bb6d7d02c9e
│       └── small_type
│           └── SmallType
│               ├── id = eb905685-23fa-4705-a14b-cea835cda321
│               ├── value = 1.0
│               └── name = Small Object
├── multiple_object_attribute
│   ├── 0
│   │   └── AnotherObject
│   │       ├── id = 6de37403-4544-4d7e-b791-7255aed0df09
│   │       └── small_type
│   │           └── SmallType
│   │               ├── id = 3a96af7e-eee1-4876-8ddf-dd097ffe8002
│   │               ├── value = 200.0
│   │               └── name = I am a small object
│   └── 1
│       └── AnotherObject
│           ├── id = 6e6be1ff-4fc0-483e-9980-5093f9ff21cb
│           └── small_type
│               └── SmallType
│                   ├── id = 1bd3eada-2beb-4598-861b-76caeefd57f2
│              

### 🎉 

Thats it! You now know how to use sdRDM to create, edit and export objects